In [1]:
import tensorflow as tf 
import numpy as np 

2024-11-07 17:44:31.827059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-07 17:44:32.009576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-07 17:44:32.072633: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 17:44:32.414448: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-07 17:44:34.603030: W tensorflow/compiler/tf2

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
train_images  = train_images.reshape(-1, 28, 28, 1).astype("float32")/255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype('float32')/255.0

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)), 
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax')

])

/home/amin/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1730988878.368402   26473 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-07 17:44:38.984016: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for yo

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=2, validation_split=0.1)


Epoch 1/2
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.8936 - loss: 0.3495 - val_accuracy: 0.9853 - val_loss: 0.0521
Epoch 2/2
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9840 - loss: 0.0524 - val_accuracy: 0.9898 - val_loss: 0.0386


In [6]:
#keras
model.save('mnist_saved_keras.keras')

In [7]:
#h5
model.save('mnist_saved_h5.h5')

In [8]:
#tflite
#Converter : Quantization (changing weights types : float32-->float16) + Prunning (removing not important connections) 

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('mnist_saved_tflite.tflite', "wb") as f :
    f.write(tflite_model)



INFO:tensorflow:Assets written to: /tmp/tmpal59ctlr/assets


INFO:tensorflow:Assets written to: /tmp/tmpal59ctlr/assets


Saved artifact at '/tmp/tmpal59ctlr'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134463014598608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134463014598032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134463014599568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134463014600336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134463014599184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134463014600144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134463014598416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134463014598992: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1730988903.873155   26473 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1730988903.873179   26473 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-11-07 17:45:03.873702: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpal59ctlr
2024-11-07 17:45:03.874143: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-11-07 17:45:03.874154: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpal59ctlr
2024-11-07 17:45:03.877550: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-11-07 17:45:03.878228: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-11-07 17:45:03.901140: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpal59ctlr
2024-11-07 17:45:03.909356: I tensorflow/cc/saved_model/loader.cc

In [9]:
#ONNX
import tf2onnx
import onnx

model.output_names = ['output']


spec =(tf.TensorSpec((None, 28, 28, 1), tf.float32, name='input'), )
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec)
onnx.save_model(onnx_model, 'mnist_saved_onnx.onnx')

I0000 00:00:1730988904.337707   26473 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730988904.342028   26473 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-11-07 17:45:04.342099: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
I0000 00:00:1730988904.342420   26473 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-07 17:45:04.344276: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned

In [10]:
#size of output models
import os


def get_file_size(file_name, file_path):
    file_size = os.path.getsize(file_path)
    file_size = file_size / (1024 * 1024)
    print(f"{file_name} size: {file_size:.2f} MB")

In [11]:
get_file_size('h5 format', 'mnist_saved_h5.h5')
get_file_size('keras format', 'mnist_saved_keras.keras')
get_file_size('onnx format', 'mnist_saved_onnx.onnx')
get_file_size('tflite format', 'mnist_saved_tflite.tflite')

h5 format size: 1.43 MB
keras format size: 1.43 MB
onnx format size: 0.47 MB
tflite format size: 0.47 MB


In [12]:
#inference_time : h5
import time
from PIL import Image


model = tf.keras.models.load_model('mnist_saved_h5.h5')
#model.summary()


def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))

    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0

    img_array = img_array.reshape(1, 28, 28, 1)

    return img_array

image_path = 'three.jpg'

start_time = time.time()

test_image = preprocess_image(image_path)

predictions = model.predict(test_image)
predicted_class = np.argmax(predictions, axis=1)

end_time = time.time()

h5_inference_time = end_time - start_time

print( 'predictied_class: ', predicted_class[0])
print("h5_inference_time", h5_inference_time)

print('FPS (frames per second): ', 1/h5_inference_time ) #---> almost realtime.(realtime > 10 FPS)





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
predictied_class:  3
h5_inference_time 0.09107685089111328
FPS (frames per second):  10.979738432058303


In [13]:
#tflite inference time

tflite_model_path = 'mnist_saved_tflite.tflite'


interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


image_path = 'three.jpg'


test_image = preprocess_image(image_path)

start_time = time.time()

interpreter.set_tensor(input_details[0]['index'], test_image)

interpreter.invoke()

predictions = interpreter.get_tensor(output_details[0]['index'])

end_time = time.time()

predicted_class = np.argmax(predictions, axis=1)
tflite_inference_time = end_time - start_time

print( 'predicted_class: ', predicted_class[0])
print("tflite_inference_time", tflite_inference_time)
print('FPS (frames per second): ', 1/tflite_inference_time )


predicted_class:  3
tflite_inference_time 0.0020296573638916016
FPS (frames per second):  492.69399741571715


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [14]:
import onnx
import onnxruntime as ort

onnx_model_path = 'mnist_saved_onnx.onnx'
onnx_model = onnx.load(onnx_model_path)


#input_shape = onnx_model.graph.input[0].type.tensor_type.shape.dim
#print(input_shape)


session = ort.InferenceSession(onnx_model_path)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


img = 'three.jpg'

start_time = time.time()

test_image = preprocess_image(img)

predictions = session.run([output_name], {input_name : test_image})

end_time = time.time()
predicted_class = np.argmax(predictions[0], axis=1)

onnx_inference_time = end_time - start_time

print( 'predicted_class: ', predicted_class[0])
print("onnx_inference_time", onnx_inference_time)
print('FPS (frames per second): ', 1/onnx_inference_time )







predicted_class:  3
onnx_inference_time 0.00586247444152832
FPS (frames per second):  170.57643661799992


In [15]:
print("h5_inference_time", h5_inference_time, 'seconds')
print("onnx_inference_time", onnx_inference_time, 'seconds')
print("tflite_inference_time", tflite_inference_time , 'seconds')

h5_inference_time 0.09107685089111328 seconds
onnx_inference_time 0.00586247444152832 seconds
tflite_inference_time 0.0020296573638916016 seconds


In [16]:
#Accuracy  --> onnx , h5 and tflite on mnist test set. 

import numpy as np
import onnx 
import onnxruntime as ort
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
from PIL import Image



(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = x_test.astype('float32')/255.0

#print(x_test.shape)


x_test = np.expand_dims(x_test, axis=-1)
#print(x_test.shape)


#evaluating h5
def evaluate_h5(h5_model_path, images):
    model = load_model(h5_model_path)
    y_pred = model.predict(images)
    return np.argmax(y_pred, axis=-1)

h5_model_path = 'mnist_saved_h5.h5'
h5_predictions = evaluate_h5(h5_model_path, x_test)
h5_accuracy = np.mean(h5_predictions==y_test)
print('h5_accuracy: ', h5_accuracy)




#evaluating tflite
def evaluate_tflite(tflite_model_path):
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()


    y_pred = []

    for i in range(len(x_test)):
        interpreter.set_tensor(input_details[0]['index'], x_test[i:i+1])
        interpreter.invoke()
        predictions = interpreter.get_tensor(output_details[0]['index'])
        y_pred.append(np.argmax(predictions))


    return np.array(y_pred)



tflite_model_path = 'mnist_saved_tflite.tflite'
tflite_predictions = evaluate_tflite(tflite_model_path)
tflite_accuracy = np.mean(tflite_predictions==y_test)

print('tflite_accuracy: ', tflite_accuracy)


#evaluating onnx
def evaluate_onnx(onnx_model_path):
    session = ort.InferenceSession(onnx_model_path)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    y_pred = []

    for i in range(len(x_test)):
        input_data = x_test[i:i+1].astype(np.float32)
        predictions = session.run([output_name], {input_name : input_data})
        y_pred.append(np.argmax(predictions[0], axis=-1))
    return np.array(y_pred).flatten()




onnx_model_path = 'mnist_saved_onnx.onnx'
onnx_predictions = evaluate_onnx(onnx_model_path)
onnx_accuracy = np.mean(onnx_predictions==y_test)

print('onnx_accuracy: ', onnx_accuracy)




313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
h5_accuracy:  0.9895
tflite_accuracy:  0.9895
onnx_accuracy:  0.9895


In [17]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader


class  SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 =  nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*7*7, 128)
        self.fc2 = nn.Linear(128, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x)
        x = x.view(x.size(0), -1) #flatten
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x 


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )

#preprocessing, transformation
transform  = transforms.Compose(
    [
        transforms.ToTensor(), 
        transforms.Normalize((0.5, ), (0.5, )) 
    ]
)


train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

model = SimpleCNN().to(device)
criterian = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterian(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch+1} , Loss: {loss.item()}' )


Epoch: 1 , Loss: 0.1588895171880722
Epoch: 2 , Loss: 0.0021344542037695646


In [19]:
#saving pt, pth, onnx

#pt
torch.save(model, 'mnist_model_pytorch.pt') #architecture, weights


#pth
torch.save(model.state_dict(), 'mnist_model_pytorch.pth') #weights

#onnx
onnx_input = torch.randn(1, 1, 28, 28).to(device)
torch.onnx.export(model, onnx_input, 'mnist_model_pytorch.onnx', input_names=['input'], output_names=['output'])

%time

CPU times: user 50 µs, sys: 2 µs, total: 52 µs
Wall time: 7.87 µs


In [20]:
get_file_size('mnist_pt_size : ', 'mnist_model_pytorch.pt')
get_file_size('mnist_pth_size : ', 'mnist_model_pytorch.pth')
get_file_size('mnist_onnx_size : ', 'mnist_model_pytorch.onnx')
get_file_size('h5 format', 'mnist_saved_h5.h5')
get_file_size('keras format', 'mnist_saved_keras.keras')
get_file_size('onnx format', 'mnist_saved_onnx.onnx')
get_file_size('tflite format', 'mnist_saved_tflite.tflite')

mnist_pt_size :  size: 1.61 MB
mnist_pth_size :  size: 1.61 MB
mnist_onnx_size :  size: 1.61 MB
h5 format size: 1.43 MB
keras format size: 1.43 MB
onnx format size: 0.47 MB
tflite format size: 0.47 MB


In [21]:
#pt inferece

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torch.load('mnist_model_pytorch.pt')
model.to(device)

model.eval()

def load_and_preprocess(image_path):
    image = Image.open(image_path).convert('L')
    preprocess = transforms.Compose([
        transforms.Resize((28, 28)), 
        transforms.ToTensor(), 
        transforms.Normalize((0.5, ), (0.5, ))
    ])
    image_tensor = preprocess(image)
    return image_tensor.unsqueeze(0)


image_path = 'three.jpg'

start_time = time.time()

input_image  = load_and_preprocess(image_path).to(device)

with torch.no_grad():
    output = model(input_image)
    predicted_class = torch.argmax(output, dim=1).item()

end_time = time.time()
pt_inference_time = end_time - start_time

print('pt predicted class: ', predicted_class)
print('pt inference time: ', pt_inference_time)

%time

pt predicted class:  3
pt inference time:  0.013625621795654297
CPU times: user 41 µs, sys: 2 µs, total: 43 µs
Wall time: 8.11 µs


/tmp/ipykernel_26473/2595358701.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('mnist_model_pytorch.pt')


In [27]:
#pth inference
import time
import torch 
import torch.nn as nn 
import torchvision.transforms as transforms
from PIL import Image 

class  SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 =  nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*7*7, 128)
        self.fc2 = nn.Linear(128, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x)
        x = x.view(x.size(0), -1) #flatten
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x 
    
def load_image(image_path):
    image = Image.open(image_path).convert('L')
    image = image.resize((28, 28))

    transform  = transforms.Compose(
    [
        transforms.ToTensor(), 
        transforms.Normalize((0.5, ), (0.5, )) 
    ]
    )

    image = transform(image)

    return image.unsqueeze(0)

def predict_pth_model(model_path, image_tensor):
    model= SimpleCNN()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)

    return predicted.item()

pth_model_path = 'mnist_model_pytorch.pth'
image_path = 'three.jpg'


start_time  = time.time()
image_tensor = load_image(image_path)
predicted_class = predict_pth_model(pth_model_path, image_tensor)
end_time = time.time()


pth_inference_time = end_time - start_time

print('pth predicted class :', predicted_class)
print('pth inference time: ', pth_inference_time)



%time

pth predicted class : 3
pth inference time:  0.027863264083862305
CPU times: user 26 µs, sys: 2 µs, total: 28 µs
Wall time: 9.06 µs


/tmp/ipykernel_26473/3435904106.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


In [ ]:
#pt , pth evaluation ---> finding performance metrics values. 
import time
import torch 
import torch.nn as nn 
import torchvision.transforms as transforms
from PIL import Image 
from torch.utils.data import DataLoader
import numpy as np 
from torchvision import datasets



transform  = transforms.Compose(
[
    transforms.ToTensor(), 
    transforms.Normalize((0.5, ), (0.5, )) 
]
)



test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class  SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 =  nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*7*7, 128)
        self.fc2 = nn.Linear(128, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x)
        x = x.view(x.size(0), -1) #flatten
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x 
    
def load_image(image_path):
    image = Image.open(image_path).convert('L')
    image = image.resize((28, 28))

    transform  = transforms.Compose(
    [
        transforms.ToTensor(), 
        transforms.Normalize((0.5, ), (0.5, )) 
    ]
    )


def evaluate_pytorch_model(model_path, model_format):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if model_format=='pth':
        model = SimpleCNN()
        model.load_state_dict(torch.load(model_path, map_location=device))
    elif model_format=='pt':
        model = torch.load(model_path, map_location=device)

    else:
        print(' input format is not defined!')

    model.to(device)
    model.eval()


    correct = 0 
    total = 0

    with torch.no_grad():  
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    

    accuracy = correct / total
    print(f'{model_format} model accuracy: ', accuracy)




evaluate_pytorch_model('mnist_model_pytorch.pt', 'pt')
evaluate_pytorch_model('mnist_model_pytorch.pth', 'pth')



/tmp/ipykernel_26473/1220161024.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


pt model accuracy:  0.988


/tmp/ipykernel_26473/1220161024.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


pth model accuracy:  0.988


In [42]:
#pytorch onnx inference
#Accuracy  --> onnx , h5 and tflite on mnist test set. 

import numpy as np
import onnx 
import onnxruntime as ort
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
from PIL import Image


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = x_test.astype('float32')/255.0

#print(x_test.shape)


x_test = np.expand_dims(x_test, axis=-1)
#print(x_test.shape)


#onnx input : 1, 1, 28, 28


#evaluating onnx
def evaluate_onnx(onnx_model_path, test_data):
    session = ort.InferenceSession(onnx_model_path)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    y_pred = []

    x_test = np.transpose(test_data, (0, 3, 1, 2)) #batch, channel, height, width
    
    for i in range(len(x_test)):
        input_data = x_test[i:i+1].astype(np.float32)
        predictions = session.run([output_name], {input_name : input_data})
        y_pred.append(np.argmax(predictions[0], axis=-1))
    return np.array(y_pred).flatten()




onnx_model_path = 'mnist_model_pytorch.onnx'
onnx_predictions = evaluate_onnx(onnx_model_path, x_test)
onnx_accuracy = np.mean(onnx_predictions==y_test)

print('onnx_accuracy: ', onnx_accuracy)





onnx_accuracy:  0.9625


In [43]:
evaluate_pytorch_model('mnist_model_pytorch.pt', 'pt')
evaluate_pytorch_model('mnist_model_pytorch.pth', 'pth')

print('onnx_accuracy: ', onnx_accuracy)

/tmp/ipykernel_26473/1220161024.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


pt model accuracy:  0.988


/tmp/ipykernel_26473/1220161024.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


pth model accuracy:  0.988
onnx_accuracy:  0.9625


In [44]:
#TENSORRT --> .engine! (will be teach in the next make up session!)

In [45]:
import numpy as np
a = np.array([[1, 2], [3, 4]])
a

array([[1, 2],
       [3, 4]])

In [46]:
np.transpose(a)

array([[1, 3],
       [2, 4]])

In [ ]:
#x_test = np.transpose(test_data, (0, 3, 1, 2))  #ijkl  ---> iljk ===> 2471--> 2147